#Luminex tSNE

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import *
from numpy import nan
# Set some Pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 25)

from sklearn.manifold import *
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import scipy.io
# import bokeh.plotting as bk
from sklearn import datasets
%matplotlib inline

##Read data

In [10]:
digits = datasets.load_digits()

datafile = 'C:/Users/wen.zhang/My Data/Luminex/TC15001 Luminex Setup JG 04Feb2015.csv'

with open(datafile) as f:
    for index, line in enumerate(f):
        if line.strip().split(',')[0] == 'DataType:' and line.strip().split(',')[1] == 'Result':
            data_start = index + 1
        elif line.strip().split(',')[0] == 'DataType:' and line.strip().split(',')[1] == 'Range':
            data_end = index - 2
print(data_start, data_end)

df = pd.read_csv(datafile, nrows=data_end - data_start, skiprows=data_start,
                 usecols=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], index_col=[0])

df.replace([r'^<', np.nan, r'^>'], [0., 0., 10000.], regex=True, inplace=True)

#df[[0,1]]
df = df.ix[24:, :]  # take off the standard and blank samples
df[[0,1]]

355 451


                        Flt-3L       GM-CSF
Sample                                     
800290 177 NALM-6   19.9845625  3070.879388
800290 177 NALM-6  21.28585746  3403.416814
Patient #1 177     23.81679958  5611.813518
Patient #1 177     18.31918275  4633.860559
177 Alone          12.22997435  114.8199577
177 Alone          12.98990876  125.9487591
800499 177 NALM-6  16.60455599  2944.667684
800499 177 NALM-6  20.63844401  2508.872863
800290 177 MEC-1   28.64409957  7370.844624
800290 177 MEC-1   25.66029502  7332.026944
Patient #1 180     23.81679958  12880.24785
Patient #1 180     22.87811327  13539.96695
...                        ...          ...
800499 180 Alone   15.90307684  284.0257565
800499 180 Alone   15.19170049  295.1175482
800499 181 MEC-1   16.60455599  489.8479109
800499 181 MEC-1   15.90307684  529.6189942
800290 181 K562     14.4696591  117.1133714
800290 181 K562    15.19170049   135.065361
PBMC 181           15.90307684  28.08707519
PBMC 181           16.60455599  

## Reformat data

In [ ]:
# Split sample names into different columns
donors = []
batch = []
target_cells = []
sample_type = []

for row in df.iterrows():
    raw_label = row[0]
    split_label = raw_label.split(" ")
    if "Patient" in raw_label:
        donors.append(split_label[0] + " " + split_label[1])
        batch.append(split_label[2])
        target_cells.append("None")
        sample_type.append("T-cells only")
        # to be continued

df['Sample type'] = sample_type
df['Donors'] = donors
df['Batch'] = batch
df['Target cell'] = target_cells

In [40]:
#df.to_csv('C:/Users/wen.zhang/My Data/Luminex/dataframe.txt')

In [19]:
numpyMatrix = np.asarray(df.as_matrix()).astype('float64')

numpyMatrix_norm = normalize(numpyMatrix, axis=0)

numpyMatrix_norm.shape

(72, 30)

## Fit Models

In [15]:
t = TSNE(n_components=2)
tsne_proj = t.fit_transform(numpyMatrix_norm)
df['TNSE1'] = tsne_proj[:, 0]
df['TNSE2'] = tsne_proj[:, 1]

isomap = Isomap(n_components=4)
isomap_proj = isomap.fit_transform(numpyMatrix_norm)
df['Isomap1'] = isomap_proj[:, 0]
df['Isomap2'] = isomap_proj[:, 1]
df['Isomap3'] = isomap_proj[:, 2]
df['Isomap4'] = isomap_proj[:, 3]

lle = LocallyLinearEmbedding(n_components=2)
lle_proj = lle.fit_transform(numpyMatrix_norm)
df['LLE1'] = lle_proj[:, 0]
df['LLE2'] = lle_proj[:, 1]

pca = PCA(n_components=4)
pca_proj = pca.fit_transform(numpyMatrix_norm)
df['PCA1'] = pca_proj[:, 0]
df['PCA2'] = pca_proj[:, 1]
df['PCA3'] = pca_proj[:, 2]
df['PCA4'] = pca_proj[:, 3]

mds = MDS(n_components=4)
mds_proj = mds.fit_transform(numpyMatrix_norm)
df['MDS1'] = mds_proj[:, 0]
df['MDS2'] = mds_proj[:, 1]
df['MDS3'] = mds_proj[:, 2]
df['MDS4'] = mds_proj[:, 3]

df.to_csv('C:/Users/wen.zhang/My Data/Luminex/dataframe_withTSNE.txt')

In [20]:
tsne_proj.shape

(72, 2)

## Plot data

In [67]:
df = pd.DataFrame({ 'x': tsne_proj[:,0],
                    'y': tsne_proj[:,1],
                    
                    'target': digits.target
                  })
src = ColumnDataSource(data=df)

fig = bk.figure(title='t-SNE - digits dataset', x_axis_label='c1', y_axis_label='c2',
                plot_width=750, plot_height=450)
fig.scatter(source=src, x='x', y='y', fill_color='color',
            size=8, line_color='black', line_alpha=0.50)

hover_tool = HoverTool(tooltips=[('target', '@target')], snap_to_data=True)
fig.add_tools(hover_tool)

bk.show(fig)

NameError: name 'digits' is not defined